# Kymograph VAE

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import numpy as np
from tqdm import tqdm
import json as json
from torch.optim import Adam

In [ ]:
"""
Import artificial kymographs
"""

datapath = r'C:/Users/Aidan/Documents/Winter_2023/BE177B/Code/data/kymoset.json'
a = np.array(json.loads(jsn))

mnist_transform = transforms.Compose([
        transforms.ToTensor(),
]) 

train_dataset = mnist_transform(datapath)
train_loader = DataLoader(train_dataset)

#maybe display one kymograph from each subgroup to double check initial categorization

In [ ]:
"""
Import experimental kymographs 
"""

#also maybe display an example to make sure it looks rights

In [ ]:
""" 
Define VAE model
Imported from Jackson Kang's notebook

TO DO:
- obtain dimensions of all our layers
    x_dim: flattened dimensionality of our input space 
    hidden_dim: dimensionality of our encoder NN
    latent_dim: dimensionality of encoded space 
- decide batch size 
- decide epoch size --> smaller probably better for runtime purposes while we are figuring stuff out

"""


### Model Hyperparameters ###
cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")
batch_size = 100   # how many images we feed each epoch

x_dim  = 784       # (flattened) dimensionality of our input space. Determined by dataset. MINST is 28x28 matrix --> 728 flattened
hidden_dim = 400   # dimensionality of our encoder NN
latent_dim = 200   # dimensionality of encoded space

lr = 1e-3 #tolerance for linear regression
epochs = 10 #number of training rounds
DEVICE = torch.device("cuda" if cuda else "cpu")


### Encoder ###
class Encoder(nn.Module): # we must define our encoder as a subclass of the provided PyTorch neural network
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def forward(self, x):
        h_       = self.LeakyReLU(self.FC_input(x))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_)                     # encoder produces mean and log of variance 
                                                       # (i.e., parateters of simple tractable normal distribution "q"
        
        return mean, log_var


### Decoder ###
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        h     = self.LeakyReLU(self.FC_hidden(x))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        
        x_hat = torch.sigmoid(self.FC_output(h))
        return x_hat


### VAE model ###
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
        
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(DEVICE)        # sampling epsilon        
        z = mean + var*epsilon                          # reparameterization trick
        return z
        
                
    def forward(self, x):
        mean, log_var = self.Encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var)
        x_hat            = self.Decoder(z)
        
        return x_hat, mean, log_var


### Instantiate model ###
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(DEVICE)


### Optimizer ###
BCE_loss = nn.BCELoss()

def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD      = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD

optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
"""" 
Train VAE model
"""